In [ ]:
import os

SEASONS = list(range(2016,2024))

DATA_DIR = "data"
STANDINGS_DIR = os.path.join(DATA_DIR, "standings")
SCORES_DIR = os.path.join(DATA_DIR, "scores")
TEMP_DIR = os.path.join(DATA_DIR, "temp")

In [ ]:
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeout
import time
# Make sure to install playwright browsers by running playwright install on the command line or !playwright install from Jupyter

In [ ]:
async def get_html(url, selector, sleep=5, retries=3):
    html = None
    for i in range(1, retries+1):
        time.sleep(sleep * i)
        try:
            async with async_playwright() as p:
                browser = await p.firefox.launch()
                page = await browser.new_page()
                await page.goto(url)
                print(await page.title())
                html = await page.inner_html(selector)
        except PlaywrightTimeout:
            print(f"Timeout error on {url}")
            continue
        else:
            break
    return html

In [ ]:
async def scrape_season(season):
    url = f"https://www.basketball-reference.com/leagues/NBA_{season}_games.html"
    html = await get_html(url, "#content .filter")
    
    soup = BeautifulSoup(html)
    links = soup.find_all("a")
    standings_pages = [f"https://www.basketball-reference.com{l['href']}" for l in links]
    
    for url in standings_pages:
        save_path = os.path.join(STANDINGS_DIR, url.split("/")[-1])
        if os.path.exists(save_path):
            continue
        
        html = await get_html(url, "#all_schedule")
        with open(save_path, "w+") as f:
            f.write(html)

In [ ]:
await scrape_season(2023)

In [ ]:
standings_files = os.listdir(STANDINGS_DIR)

In [ ]:
async def scrape_game(standings_file):
    with open(standings_file, 'r') as f:
        html = f.read()

    soup = BeautifulSoup(html)
    links = soup.find_all("a")
    hrefs = [l.get('href') for l in links]
    box_scores = [f"https://www.basketball-reference.com{l}" for l in hrefs if l and "boxscore" in l and '.html' in l]

    for url in box_scores:
        save_path = os.path.join(TEMP_DIR, url.split("/")[-1])
        check_path_1 = os.path.join(SCORES_DIR, url.split("/")[-1])
        check_path_2 = os.path.join(TEMP_DIR, url.split("/")[-1])
        if os.path.exists(check_path_1) or os.path.exists(check_path_2):
            continue

        html = await get_html(url, "#content")
        if not html:
            continue
        with open(save_path, "w+") as f:
            f.write(html)

In [ ]:
import pandas as pd
SEASONS = range(2022,2024)
for season in SEASONS:
    files = [s for s in standings_files if str(season) in s]
    
    for f in files:
        filepath = os.path.join(STANDINGS_DIR, f)
        
        await scrape_game(filepath)

In [ ]:
files = [s for s in standings_files if str(season) in s]
for f in files:
    filepath = os.path.join(STANDINGS_DIR, f)
        
    await scrape_game(filepath)

In [ ]:
import os
import pandas as pd

SCORE_DIR = "data/scores"

In [ ]:
box_scores = os.listdir(SCORE_DIR)
box_scores = [os.path.join(SCORE_DIR, f) for f in box_scores if f.endswith(".html")]

In [ ]:
from bs4 import BeautifulSoup

def parse_html(box_score):
    with open(box_score) as f:
        html = f.read()

    soup = BeautifulSoup(html)
    [s.decompose() for s in soup.select("tr.over_header")]
    [s.decompose() for s in soup.select("tr.thead")]
    return soup

In [ ]:
def read_season_info(soup):
    nav = soup.select("#bottom_nav_container")[0]
    hrefs = [a["href"] for a in nav.find_all('a')]
    season = os.path.basename(hrefs[1]).split("_")[0]
    return season

In [ ]:
def read_line_score(soup):
    line_score = pd.read_html(str(soup), attrs = {'id': 'line_score'})[0]
    cols = list(line_score.columns)
    cols[0] = "team"
    cols[-1] = "total"
    line_score.columns = cols
    
    line_score = line_score[["team", "total"]]
    
    return line_score

In [ ]:
def read_line_score_test(soup):
    teams = soup.select('.scorebox')[0]
    hrefs = [t["href"] for t in teams.find_all('a')]
    team = [t for t in hrefs if '/teams' in t]
    first_team = team[0].split('/')[2]
    second_team = team[1].split('/')[2]
    scores = soup.find_all('div',{'class':'scores'})
    first_score = int(scores[0].text)
    second_score =int(scores[1].text)
    return pd.DataFrame({'team':[first_team,second_team],'total':[first_score,second_score]})
    

In [ ]:
def read_stats(soup, team, stat):
    df = pd.read_html(str(soup), attrs = {'id': f'box-{team}-game-{stat}'}, index_col=0)[0]
    df = df.apply(pd.to_numeric, errors="coerce")
    return df

In [ ]:
games = []
base_cols = None
for box_score in box_scores:
    soup = parse_html(box_score)

    line_score = read_line_score_test(soup)
    teams = list(line_score["team"])

    summaries = []
    for team in teams:
        basic = read_stats(soup, team, "basic")
        advanced = read_stats(soup, team, "advanced")

        totals = pd.concat([basic.iloc[-1,:], advanced.iloc[-1,:]])
        totals.index = totals.index.str.lower()

        maxes = pd.concat([basic.iloc[:-1].max(), advanced.iloc[:-1].max()])
        maxes.index = maxes.index.str.lower() + "_max"

        summary = pd.concat([totals, maxes])
        
        if base_cols is None:
            base_cols = list(summary.index.drop_duplicates(keep="first"))
            base_cols = [b for b in base_cols if "bpm" not in b]
        
        summary = summary[base_cols]
        
        summaries.append(summary)
    summary = pd.concat(summaries, axis=1).T

    game = pd.concat([summary, line_score], axis=1)

    game["home"] = [0,1]

    game_opp = game.iloc[::-1].reset_index()
    game_opp.columns += "_opp"

    full_game = pd.concat([game, game_opp], axis=1)
    full_game["season"] = read_season_info(soup)
    
    full_game["date"] = os.path.basename(box_score)[:8]
    full_game["date"] = pd.to_datetime(full_game["date"], format="%Y%m%d")
    
    full_game["won"] = full_game["total"] > full_game["total_opp"]
    games.append(full_game)
    
    if len(games) % 100 == 0:
        print(f"{len(games)} / {len(box_scores)}")

In [ ]:
games_df = pd.concat(games, ignore_index=True)

In [ ]:
games_df.to_csv("nba_games_updated.csv")

# Run following cells to update data, change month as needed

In [1]:
import os
import pandas as pd

SEASONS = list(range(2016,2024))

DATA_DIR = "data"
STANDINGS_DIR = os.path.join(DATA_DIR, "standings")
SCORES_DIR = os.path.join(DATA_DIR, "scores")
TEMP_DIR = os.path.join(DATA_DIR, "temp")
standings_files = os.listdir(STANDINGS_DIR)

from bs4 import BeautifulSoup
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeout
import time
# Make sure to install playwright browsers by running playwright install on the command line or !playwright install from Jupyter

async def get_html(url, selector, sleep=5, retries=3):
    html = None
    for i in range(1, retries+1):
        time.sleep(sleep * i)
        try:
            async with async_playwright() as p:
                browser = await p.firefox.launch()
                page = await browser.new_page()
                await page.goto(url)
                print(await page.title())
                html = await page.inner_html(selector)
        except PlaywrightTimeout:
            print(f"Timeout error on {url}")
            continue
        else:
            break
    return html

async def scrape_season(season):
    url = f"https://www.basketball-reference.com/leagues/NBA_{season}_games.html"
    html = await get_html(url, "#content .filter")
    
    soup = BeautifulSoup(html)
    links = soup.find_all("a")
    standings_pages = [f"https://www.basketball-reference.com{l['href']}" for l in links]
    
    for url in standings_pages:
        save_path = os.path.join(STANDINGS_DIR, url.split("/")[-1])
        if os.path.exists(save_path):
            continue
        
        html = await get_html(url, "#all_schedule")
        with open(save_path, "w+") as f:
            f.write(html)
            
async def scrape_game(standings_file):
    with open(standings_file, 'r') as f:
        html = f.read()

    soup = BeautifulSoup(html)
    links = soup.find_all("a")
    hrefs = [l.get('href') for l in links]
    box_scores = [f"https://www.basketball-reference.com{l}" for l in hrefs if l and "boxscore" in l and '.html' in l]

    for url in box_scores:
        save_path = os.path.join(TEMP_DIR, url.split("/")[-1])
        check_path_1 = os.path.join(SCORES_DIR, url.split("/")[-1])
        check_path_2 = os.path.join(TEMP_DIR, url.split("/")[-1])
        if os.path.exists(check_path_1) or os.path.exists(check_path_2):
            continue

        html = await get_html(url, "#content")
        if not html:
            continue
        with open(save_path, "w+") as f:
            f.write(html)
            
def parse_html(box_score):
    with open(box_score) as f:
        html = f.read()

    soup = BeautifulSoup(html)
    [s.decompose() for s in soup.select("tr.over_header")]
    [s.decompose() for s in soup.select("tr.thead")]
    return soup

def read_season_info(soup):
    nav = soup.select("#bottom_nav_container")[0]
    hrefs = [a["href"] for a in nav.find_all('a')]
    season = os.path.basename(hrefs[1]).split("_")[0]
    return season

def read_line_score(soup):
    line_score = pd.read_html(str(soup), attrs = {'id': 'line_score'})[0]
    cols = list(line_score.columns)
    cols[0] = "team"
    cols[-1] = "total"
    line_score.columns = cols
    
    line_score = line_score[["team", "total"]]
    
    return line_score

def read_stats(soup, team, stat):
    df = pd.read_html(str(soup), attrs = {'id': f'box-{team}-game-{stat}'}, index_col=0)[0]
    df = df.apply(pd.to_numeric, errors="coerce")
    return df

def read_line_score_test(soup):
    teams = soup.select('.scorebox')[0]
    hrefs = [t["href"] for t in teams.find_all('a')]
    team = [t for t in hrefs if '/teams' in t]
    first_team = team[0].split('/')[2]
    second_team = team[1].split('/')[2]
    scores = soup.find_all('div',{'class':'scores'})
    first_score = int(scores[0].text)
    second_score =int(scores[1].text)
    return pd.DataFrame({'team':[first_team,second_team],'total':[first_score,second_score]})
    



In [2]:
## Delete old standings file and scrape new with most recent scores
os.remove('data/standings/NBA_2023_games-january.html')  ##Update month as needed
await scrape_season(2023)

2022-23 NBA Schedule | Basketball-Reference.com
Timeout error on https://www.basketball-reference.com/leagues/NBA_2023_games-january.html
2022-23 NBA Schedule | Basketball-Reference.com


In [3]:
## run scrape_game function which opens each box score and saves it as an html file - skips if file already exists
for season in SEASONS:
    files = [s for s in standings_files if str(season) in s]
    
    for f in files:
        filepath = os.path.join(STANDINGS_DIR, f)
        
        await scrape_game(filepath)

Clippers vs Mavericks, January 22, 2023 | Basketball-Reference.com
Pelicans vs Heat, January 22, 2023 | Basketball-Reference.com
Knicks vs Raptors, January 22, 2023 | Basketball-Reference.com
Thunder vs Nuggets, January 22, 2023 | Basketball-Reference.com
Timeout error on https://www.basketball-reference.com/boxscores/202301220PHO.html
Grizzlies vs Suns, January 22, 2023 | Basketball-Reference.com
Nets vs Warriors, January 22, 2023 | Basketball-Reference.com
Lakers vs Trail Blazers, January 22, 2023 | Basketball-Reference.com
Bucks vs Pistons, January 23, 2023 | Basketball-Reference.com
Celtics vs Magic, January 23, 2023 | Basketball-Reference.com
Hawks vs Bulls, January 23, 2023 | Basketball-Reference.com
Timberwolves vs Rockets, January 23, 2023 | Basketball-Reference.com
Hornets vs Jazz, January 23, 2023 | Basketball-Reference.com
Timeout error on https://www.basketball-reference.com/boxscores/202301230POR.html
Spurs vs Trail Blazers, January 23, 2023 | Basketball-Reference.com
Griz

In [5]:
box_scores = os.listdir(TEMP_DIR)
box_scores = [os.path.join(TEMP_DIR, f) for f in box_scores if f.endswith(".html")]
games = []
base_cols = None
for box_score in box_scores:
    soup = parse_html(box_score)

    line_score = read_line_score_test(soup)
    teams = list(line_score["team"])

    summaries = []
    for team in teams:
        basic = read_stats(soup, team, "basic")
        advanced = read_stats(soup, team, "advanced")

        totals = pd.concat([basic.iloc[-1,:], advanced.iloc[-1,:]])
        totals.index = totals.index.str.lower()

        maxes = pd.concat([basic.iloc[:-1].max(), advanced.iloc[:-1].max()])
        maxes.index = maxes.index.str.lower() + "_max"

        summary = pd.concat([totals, maxes])
        
        if base_cols is None:
            base_cols = list(summary.index.drop_duplicates(keep="first"))
            base_cols = [b for b in base_cols if "bpm" not in b]
        
        summary = summary[base_cols]
        
        summaries.append(summary)
    summary = pd.concat(summaries, axis=1).T

    game = pd.concat([summary, line_score], axis=1)

    game["home"] = [0,1]

    game_opp = game.iloc[::-1].reset_index()
    game_opp.columns += "_opp"

    full_game = pd.concat([game, game_opp], axis=1)
    full_game["season"] = read_season_info(soup)
    
    full_game["date"] = os.path.basename(box_score)[:8]
    full_game["date"] = pd.to_datetime(full_game["date"], format="%Y%m%d")
    
    full_game["won"] = full_game["total"] > full_game["total_opp"]
    games.append(full_game)
    
    if len(games) % 100 == 0:
        print(f"{len(games)} / {len(box_scores)}")

In [6]:
old = pd.read_csv("nba_games_updated.csv",index_col=0)

In [7]:
old.head()

,mp,mp.1,fg,fga,fg%,3p,3pa,3p%,ft,fta,...,tov%_max_opp,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,won
0,240.0,240.0,38.0,72.0,0.528,16.0,37.0,0.432,18.0,21.0,...,26.2,26.8,155.0,123.0,MIA,107,1,2023,2022-11-07,True
1,240.0,240.0,40.0,84.0,0.476,14.0,39.0,0.359,13.0,15.0,...,41.0,37.3,160.0,121.0,POR,110,0,2023,2022-11-07,False
2,240.0,240.0,41.0,78.0,0.526,8.0,24.0,0.333,15.0,19.0,...,28.6,41.1,250.0,125.0,DAL,90,1,2023,2022-12-14,True
3,240.0,240.0,29.0,74.0,0.392,13.0,38.0,0.342,19.0,26.0,...,12.6,33.0,183.0,110.0,CLE,105,0,2023,2022-12-14,False
4,240.0,240.0,39.0,81.0,0.481,6.0,20.0,0.300,14.0,18.0,...,22.8,29.0,178.0,111.0,DAL,95,1,2016,2015-12-09,True


In [8]:
temp_df = pd.concat(games, ignore_index=True)
temp_df.to_csv("temp.csv")
temp = pd.read_csv("temp.csv",index_col=0)
old = pd.read_csv("nba_games_updated.csv",index_col=0)
save = pd.concat([old,temp])
save.team = save.team.str.replace('CHO','CHA').str.replace('PHO','PHX').str.replace('BRK','BKN').str.replace('NJN','BKN').str.replace('NOH','NOP')
save.to_csv("nba_games_updated.csv")

# gather all files
allfiles = os.listdir(TEMP_DIR)
 
# iterate on all files to move them to destination folder
for f in allfiles:
    src_path = os.path.join(TEMP_DIR, f)
    dst_path = os.path.join(SCORES_DIR, f)
    os.rename(src_path, dst_path)

In [9]:
save.tail()

,mp,mp.1,fg,fga,fg%,3p,3pa,3p%,ft,fta,...,tov%_max_opp,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,won
23,240.0,240.0,43.0,84.0,0.512,11.0,27.0,0.407,16.0,24.0,...,100.0,34.2,155.0,126.0,BOS,98,0,2023,2023-01-23,True
24,240.0,240.0,35.0,77.0,0.455,12.0,28.0,0.429,30.0,31.0,...,20.2,37.4,277.0,134.0,DAL,98,1,2023,2023-01-22,True
25,240.0,240.0,35.0,80.0,0.438,13.0,35.0,0.371,15.0,26.0,...,25.6,28.3,157.0,117.0,LAC,112,0,2023,2023-01-22,False
26,240.0,240.0,38.0,85.0,0.447,2.0,16.0,0.125,24.0,34.0,...,66.7,29.6,228.0,109.0,UTA,120,1,2023,2023-01-23,False
27,240.0,240.0,41.0,86.0,0.477,16.0,40.0,0.400,22.0,26.0,...,31.3,31.3,200.0,125.0,CHO,102,0,2023,2023-01-23,True
